In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

import sys
# -----------------------------
# Ajouter le repo au Python Path
# -----------------------------
sys.path.append("/Workspace/Users/mandu543@gmail.com/databricks_flights")

from src.Communs.utils import *

In [0]:
df = spark.sql(f"SELECT * FROM {SILVER_ZONE}.silver_flights_flights")
display(df)

## Créer une alimentation Dynamique vers Gold Layer

#### Parameters

Pas besoin de paramètres car multi trading et non loop 

In [0]:
# Key Cols List
key_cols_list = "['flight_id']" # eval transforme en liste
key_cols_list = eval(key_cols_list)
# CDC Column
cdc_column = "modifiedDate"

# BACK-DATED REFRESH
back_dated_refresh = ""

# Source Object 
source_object = "silver_flights_flights"

# Source Schema
source_schema = SILVER_ZONE

# Target Object
target_object = "fact_flights_flights"

# Targer Schema 
target_schema = GOLD_ZONE

# Name Col Surrogate Key
colSurrogateKey = "DimSurrogateKey"


## INCREMENTAL DATA INGESTION

### Last Load Date

In [0]:
# Without backdated refresh
if len(back_dated_refresh) == 0:
  
  # If Table Destination exists => get last load date
  if spark.catalog.tableExists(f"{target_schema}.{target_object}") :

    last_load = spark.sql(f"SELECT max({cdc_column}) FROM {target_schema}.{target_object}").collect()[0][0]

  else: 

    last_load = "1900-01-01 00:00:00"

# backdated refresh exists   
else : 
  last_load = back_dated_refresh

# test result 
last_load

In [0]:
# Read Data After Specific Date
df_source = spark.sql(f"SELECT * FROM {source_schema}.{source_object} WHERE {cdc_column} >= '{last_load}'")

In [0]:
df_source.display()

### OLD VS NEW RECORDS

In [0]:
if spark.catalog.tableExists(f"{target_schema}.{target_object}") :
    
    # Key Columns String For Incremental
    key_cols_string_incremental = ', '.join(key_cols_list)
    # Data Already Exists
    df_target = spark.sql(f"SELECT {key_cols_string_incremental}, {colSurrogateKey}, create_date, update_date FROM {target_schema}.{target_object}")

else :

    # Key Columns String For Initial
    key_cols_string_initial = [f"'' as {i}" for i in key_cols_list]
    key_cols_string_initial = ', '.join(key_cols_string_initial)
    key_cols_string_initial

    df_target = spark.sql(f"SELECT {key_cols_string_initial}, 0 as {colSurrogateKey}, CAST('1900-01-01 00:00:00' AS timestamp) as create_date, cast('1900-01-01 00:00:00' AS timestamp) as update_date WHERE 1=0")

df_target.display()

In [0]:
df_source.createOrReplaceTempView("src")
df_target.createOrReplaceTempView("target")

join_condition = ' AND '.join([f"src.{i} = target.{i}" for i in key_cols_list]) # Join Condition

df_join = spark.sql(f"""
                    SELECT src.*,
                            target.{colSurrogateKey}, 
                            target.create_date, 
                            target.update_date 
                    FROM src 
                    LEFT JOIN target 
                    ON {join_condition} 
                """)

In [0]:
df_join.display()

In [0]:
# DF OLD RECORDS
df_old = df_join.filter(col(f"{colSurrogateKey}").isNotNull())
df_old.display()

# DF NEW RECORDS
df_new = df_join.filter(col(f"{colSurrogateKey}").isNull())
df_new.display()


##ENRICHING DATAFRAMES

### PREPARING DF_OLD

In [0]:
df_olf_record = df_old.withColumn("update_date", lit(current_timestamp()))

### **PREPARING DF_NEW**

In [0]:
if spark.catalog.tableExists(f"{target_schema}.{target_object}") :
    # Update Data For Update 
   max_colSurrogateKey = spark.sql(f"select max({colSurrogateKey}) as max_value from {target_schema}.{target_object}").collect()[0]['max_value']
   df_new_record = df_new.withColumn(f'{colSurrogateKey}', lit(max_colSurrogateKey)+ lit(1)+ monotonically_increasing_id()) \
                .withColumn("update_date", current_timestamp()) \
                .withColumn("create_date", current_timestamp())
else : 

    # New Data For Inssertion 
    max_colSurrogateKey = 0
    df_new_record = df_new.withColumn(f'{colSurrogateKey}', lit(max_colSurrogateKey)+ lit(1)+ monotonically_increasing_id()) \
                            .withColumn("create_date", current_timestamp()) \
                            .withColumn("update_date", current_timestamp())
                 
    
    

In [0]:
df_new_record.display()
df_olf_record.display()



## UNION OLD AND NEW RECORDS

In [0]:
df_union = df_olf_record.unionByName(df_new_record)
df_union.display()


## **UPSET**

In [0]:
if spark.catalog.tableExists(f"{target_schema}.{target_object}") :
    pass
else :
    df_union.write.format("delta")\
        .mode("append")\
            .option("path")
            .saveAsTable(f"{target_schema}.{target_object}")